In [ ]:
import sys
import os
import itertools
from tempfile import NamedTemporaryFile
from gc import collect
from time import time

import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr

sys.path.append('../util')
from ncgen import make_nc
from meters import ThroughputMeter
from circle import wkt_circle
from geo import cartesian
from areas import metro_van_10, prov_bc_18, fp_270
from grids import bc_400m, canada_5k, world_125k, world_250k

from shapely import wkt
from shapely.geometry import Point, Polygon, box


os.environ['TMPDIR'] = os.getenv('HOME') + '/tmp/'

In [ ]:
def clipLatLonToPolyExtent(lat, lon, poly):
    minx, miny, maxx, maxy = poly.bounds    
    lat = lat[np.where( (lat > miny) & (lat < maxy) )]
    lon = lon[np.where( (lon > minx) & (lon < maxx) )]
    
    return lat, lon

def pointsInPoly(coords, poly):    
    x = [1 if poly.contains(Point(p[0], p[1])) else 0 for p in coords]
    coords = coords[np.where(x)]
    return coords

In [ ]:
def polygonToMask(nc, poly):
    nclats = nc.variables['lat'][:]
    nclons = nc.variables['lon'][:]

    mask = np.zeros(nc.variables['var_0'][0,:,:].shape)
    
    lat, lon = clipLatLonToPolyExtent(nclats, nclons, poly)
    
    # If no centroids in polygon, return zeros
    if lat.size == 0 or lon.size == 0:
        return mask

    coords = cartesian([lon, lat])
    pts = pointsInPoly(coords, poly)

    # map the lon, lat values back to numpy indices
    lons = np.sort(np.unique(pts[:,0]))
    lats = np.sort(np.unique(pts[:,1]))
    lon_indices = nclons.searchsorted(lons)
    lat_indices = nclats.searchsorted(lats)

    lat_map = dict(zip(lats, lat_indices))
    lon_map = dict(zip(lons, lon_indices))
    

    for pt in pts:
        mask[lon_map[pt[0]],lat_map[pt[1]]] = 1
        
    return mask

In [ ]:
from rtree import index

class GriddedPolygon:
    '''
    This builds a queryable structure for high speed point in polygon operations.
    Based on partitioning the polygon into an n*n grid
    '''
    def __init__(self, poly, size=12):
        assert isinstance(poly, Polygon)
        self.poly = poly
        self.minx, self.miny, self.maxx, self.maxy = poly.bounds
        self.size = size
        self.interior_idx = index.Index()
        self.exterior_idx = index.Index()
        
        self.grid = self._generate_grid()
        self._index_grid()

    def _generate_grid(self):
        x_range = np.linspace(self.minx, self.maxx, self.size + 1)
        y_range = np.linspace(self.miny, self.maxy, self.size + 1)
        
        grid = []
        for xi in range(self.size):
            for yi in range(self.size):
                grid.append(box(x_range[xi], y_range[yi], x_range[xi + 1], y_range[yi] + 1))
        return grid

    def _index_grid(self):
        '''
        Both shapely and Rtree use logic that a point on the boundary of an object are not 
        contained within that object. This could lead to points on the boundary of known
        inside/ouside grid cell's being needlessly tested.
        '''
        for i, g in enumerate(self.grid):
            if poly.contains(g):
                self.interior_idx.insert(i, g.bounds)
            elif poly.disjoint(g):
                self.exterior_idx.insert(i, g.bounds)

    def contains(self, p):
        '''
        First test for exerior index, then interior, then edges
        '''
        assert isinstance(p, Point)
        x, y = p.x, p.y

        # Check known interior
        try:
            next(self.interior_idx.intersection((x, y, x, y)))
            return True
        except StopIteration:
            pass

        # Check known exterior
        try:
            next(self.exterior_idx.intersection((x, y, x, y)))
            return False
        except StopIteration:
            pass

        # Check unknown area
        if Point(x, y).within(self.poly):
            return True

        return False

def polygonToMaskGrid(nc, poly):
    # Grid method: http://erich.realtimerendering.com/ptinpoly/
    nclats = nc.variables['lat'][:]
    nclons = nc.variables['lon'][:]

    mask = np.zeros(nc.variables['var_0'][0,:,:].shape)

    lat, lon = clipLatLonToPolyExtent(nclats, nclons, poly)
    
    # If no centroids in polygon, return zeros
    if lat.size == 0 or lon.size == 0:
        return mask

    coords = cartesian([lon, lat])
    grid_poly = GriddedPolygon(poly)
    pts = pointsInPoly(coords, grid_poly)

    # map the lon, lat values back to numpy indices
    lons = np.sort(np.unique(pts[:,0]))
    lats = np.sort(np.unique(pts[:,1]))
    lon_indices = nclons.searchsorted(lons)
    lat_indices = nclats.searchsorted(lats)

    lat_map = dict(zip(lats, lat_indices))
    lon_map = dict(zip(lons, lon_indices))

    for pt in pts:
        mask[lon_map[pt[0]],lat_map[pt[1]]] = 1

    return mask


In [ ]:
poly_complexity = [8, 16, 32, 64, 128]
grids = [world_250k, world_125k, canada_5k, bc_400m]


In [ ]:
from rtree import index

idx = index.Index()
idx.insert(1, (-118.8720703125, 49.12711546919731, -117.18017578125, 50.12711546919731))
if list(idx.intersection((-118, 50, -118, 50))):
    print('intersect')

it = idx.intersection((-118, 50, -118, 50))
try:
    next(idx.intersection((-120, 50, -120, 50)))
    print('intersect')
except StopIteration:
    print('nope')


In [ ]:
results = []
grid_results = []

for grid in grids:
    with NamedTemporaryFile(suffix='.nc', delete=False, dir='/app/tmp') as f:
        nc = make_nc(f.name, grid=grid)
        
        # for sides in poly_complexity: #Circle generator doesn't seem to be working... circle centers near equator
        for area in metro_van_10, prov_bc_18, fp_270:
#           poly = wkt.loads(wkt_circle(sides))
            poly = wkt.loads(area)

            lat, lon = clipLatLonToPolyExtent(nc.variables['lat'][:], nc.variables['lon'][:], poly)
            if lat.size == 0 or lon.size == 0:
                pts_tested = 0
            else:
                pts_tested = len(lon) * len(lat)
           
            t0 = time()
            mask = polygonToMaskGrid(nc, poly)
            t1 = time()
            t = t1 - t0

            r = (len(poly.exterior.coords), pts_tested, np.sum(mask), t)
            print(r)
            grid_results.append(r)

            t0 = time()
            mask = polygonToMask(nc, poly)
            t1 = time()
            t = t1 - t0
            r = (len(poly.exterior.coords), pts_tested, np.sum(mask), t)
            print(r)
            results.append(r)

In [ ]:
gra = np.array([(r[1], r[3]) for r in grid_results])
ra = np.array([(r[1], r[3]) for r in results])

In [ ]:
%matplotlib inline
plt.plot(ra[:,0], ra[:,1], 'r.', label="ungridded")
plt.plot(gra[:,0], gra[:,1], 'r-', label="gridded")

plt.xlabel('Points tested')
plt.ylabel('Time (s)')